### 練習問題2.70
以下に⽰す8記号の相対頻度つきアルファベットは、  
1950年代のロックの歌詞を効率よく符号化できるよう設計されたものである  
(“アルファベット”の“記号”が単独の⽂字であるとは限らないことに注意)。  

    A 2
    GET 2
    SHA 3
    WAH 1
    BOOM 1
    JOB 2
    NA 16
    YIP 9

generate-huffman-tree(練習問題2.69)を使って対応するハフマ ン木を⽣成し、  
encode (練習問題2.68)を使って次のメッセージを符号化せよ。  

    Get a job
    Sha na na na na na na na na
    Get a job
    Sha na na na na na na na na
    Wah yip yip yip yip yip yip yip yip yip
    Sha boom  

符号化には何ビット必要だろうか。  
もし固定⻑符号をこの8記号アルファベットに使ったとしたら、  
最低でどれだけのビット数が必要になるだろうか。

In [1]:
(define (make-leaf symbol weight) (list 'leaf symbol weight))
(define (leaf? object) (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))

; コンストラクタ
(define (make-code-tree left right)
  (list left right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))
    )
  )

; セレクタ
(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree) (list (symbol-leaf tree))
      (caddr tree)
    )
  )
(define (weight tree)
  (if (leaf? tree) (weight-leaf tree)
      (cadddr tree)
    )
  )

(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits) '()
        (let ((next-branch (choose-branch (car bits) current-branch)))
          (if (leaf? next-branch) (cons (symbol-leaf next-branch) (decode-1 (cdr bits) tree))
              (decode-1 (cdr bits) next-branch))
          )
    )
    )
  (decode-1 bits tree)
)  
(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit: CHOOSE-BRANCH" bit))
        )
  )

(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set) (adjoin-set x (cdr set))))))

(define (make-leaf-set pairs)
  (if (null? pairs) '()
      (let ((pair (car pairs)))
        (adjoin-set (make-leaf (car pair) ; symbol
                               (cadr pair)) ; weight
                    (make-leaf-set (cdr pairs)))
        )
    )
  )

In [2]:
; 符号化処理
(define (encode-symbol symbol tree)
  (define (iter sub result)
    (if (leaf? sub)
        (if (eq? (symbol-leaf sub) symbol) result
            '())
        (let ((l (left-branch sub))
              (r (right-branch sub)))
          (let ((l-result (iter l (append result '(0)))))
            (if (not (null? l-result)) l-result
                (iter r (append result '(1)))
              )
            )
        )
      )
    )
  (let ((result (iter tree '())))
    (if (null? result) (error "bad symbol: ENCODE" symbol)
        result
        )
    )
)

; 符号化処理 
(define (encode message tree)
    (if (null? message) '()
        (append
            (encode-symbol (car message) tree)
            (encode (cdr message) tree))
    )
)

In [3]:
; ハフマン符号木の生成の記号・頻度ペアのリストから、指定したペアを削除したリストを返す。
(define (remove-set item set)
  (if (null? set) '()
      (if (equal? item (car set)) (remove-set item (cdr set))
          (cons (car set) (remove-set item (cdr set)))
      )
  )
)

; 回答
(define (successive-merge set)
  (define (iter result)
    (cond ((null? result) '())
          ((= (length result) 1) (car result))
          (else
             (let ((top (car result))
                   (next (cadr result))
                   )
               (let (
                     (new-set (remove-set next (remove-set top result)))
                     ;(new-set (cddr result)) ; remove-set手続きを使わなくてもcddrで代用可能
                     )
                   (iter (adjoin-set (make-code-tree top next) new-set))
                 )
               )
           )
        )
    )
  (iter set)
  )

(define (generate-huffman-tree pairs)
    (successive-merge (make-leaf-set pairs)))

In [4]:
;(define pairs '((na 32) (yip 16) (get 9) (a 2) (job 2) (wah 1) (sha 1) (boom 1)))
(define pairs '((na 16) (yip 9) (sha 3) (get 2) (a 2) (job 2) (wah 1) (boom 1)))
(define sample (generate-huffman-tree pairs))
sample

((leaf na 16) ((leaf yip 9) (((leaf get 2) ((leaf boom 1) (leaf wah 1) (boom wah) 2) (get boom wah) 4) ((leaf sha 3) ((leaf job 2) (leaf a 2) (job a) 4) (sha job a) 7) (get boom wah sha job a) 11) (yip get boom wah sha job a) 20) (na yip get boom wah sha job a) 36)

In [5]:
; 単一の記号で動作確認
(display (encode '(na) sample))
(newline)
(display (encode '(yip) sample))
(newline)
(display (encode '(sha) sample))
(newline)
(display (encode '(get) sample))
(newline)
(display (encode '(a) sample))
(newline)
(display (encode '(job) sample))
(newline)
(display (encode '(wah) sample))
(newline)
(display (encode '(boom) sample))
(newline)

(0)
(1 0)
(1 1 1 0)
(1 1 0 0)
(1 1 1 1 1)
(1 1 1 1 0)
(1 1 0 1 1)
(1 1 0 1 0)


In [6]:
(define message '(get a job sha na na na na na na na na get a job sha na na na na na na na na wah yip yip yip yip yip yip yip yip yip sha boom))
(length message)

36

In [7]:
(define encode-bits (encode message sample))
encode-bits

(1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1 0 1 0)

In [8]:
; エンコード後のビット長
(length encode-bits)

84

In [9]:
(decode encode-bits sample)

(get a job sha na na na na na na na na get a job sha na na na na na na na na wah yip yip yip yip yip yip yip yip yip sha boom)

固定符号の場合、  
8記号あるため、  
2^3=8で1記号あたり3ビット必要。

86語であるため、86\*3=258bit必要。